# 1) 패키지 설치 및 OpenAI 키 값 설

In [1]:
!pip install langchain_openai langchain_community langchain_chroma pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 66.7 MB/s eta 0:00

In [2]:
import os
import urllib.request
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

# 2) 데이터 다운로드

In [4]:
# 분석할 PDF 파일을 웹에서 다운로드.
urllib.request.urlretrieve("https://github.com/llama-index-tutorial/llama-index-tutorial/raw/main/ch07/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf", filename="2023_북한인권보고서.pdf")

('2023_북한인권보고서.pdf', <http.client.HTTPMessage at 0x7c479252e8d0>)

# 3) 거대 언어 모델과 임베딩 설정

In [6]:
#Langchain의 LLM과 임베딩 모델 설정
llm = ChatOpenAI(model = 'gpt-4o', temperature= 0.2)
embed_model = OpenAIEmbeddings(model = 'text-embedding-3-large')

#문서 분할 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap =100)

#PDF를 읽고 벡터 인덱스 생성
loader= PyPDFLoader("2023_북한인권보고서.pdf")
documents = loader.load()
chunks = text_splitter.split_documents(documents)
vector_score = Chroma.from_documents(chunks, embed_model)

# 4) 하이드 구현하기

In [7]:
def generate_hypothetical_doc(question : str) -> str:
    """질문에 대한 가상의 이상적인 답변 문서 생성"""
    prompt = f""" 주어진 질문에 대해, 마치 실제 문서에서 발췌한 것 갗은 이상적인 답변을 작성해주세요.
    단, 구체적인 수치, 날짜, 트렌드와 같은 상세 정보를 포함해야 합니다.

    질문: {question}

    답변:"""

    response = llm.invoke(prompt)
    return response.content

In [9]:
def search_with_hyde(hypertical_doc: str):
    """가상 문서를 이용해 실제 문서 검색"""
    results = vector_score.similarity_search_with_score(hypertical_doc, k=4)
    return [
        {
            'content' : doc.page_content,
            'score': score
        } for doc, score in results
    ]

In [10]:
def generate_final_answer(question: str, relevant_docs: list) -> str:
    """ 검색된 문서를 바탕으로 최종 답변 생성"""
    context = "\n\n".join(doc['content'] for doc in relevant_docs)

    prompt = f"""다음 검색 결과를 바탕으로 질문에 답변하세요.
    검색 결과의 정보를 최대한 사용하고, 없는 정보는 답변하지 마세요.

    검색 결과:
    {context}

    질문: {question}

    답변 : """

    response = llm.invoke(prompt)

    return response.content

In [16]:
def process_query(question : str):
    """ 전체 HyDE 프로세스"""
    #1. 가상 문서 생성
    print("1. 가상 문서 생성중...")
    hypothetical_doc = generate_hypothetical_doc(question)
    print("\n 가상 문서:", hypothetical_doc)

    #2. 가상 문서로 검색
    print("\n2. 실제 문서 검색중...")
    relevant_docs = search_with_hyde(hypothetical_doc)

    #3. 최종 답변 생성
    print("\n3. 최종 답변 생성중...")
    final_answer = generate_final_answer(question, relevant_docs)

    return {
        "hypothetical_doc": hypothetical_doc,
        "retrieved_docs": relevant_docs,
        "final_answer": final_answer
    }

In [17]:
question = "북한에서 강제로 이루어지는 조직 생활은 무엇인가요."
result = process_query(question)

print("\n=== 프로세스 결과 ===")
print("\n[가상 문서]")
print(result["hypothetical_doc"])
print("\n[검색된 문서들]")
for idx, doc in enumerate(result["retrieved_docs"], 1):
    print(f"\n문서 {idx} (유사도 점수: {doc['score']:.4f}):")
    print(doc['content'])
print("\n[최종 답변]")
print(result["final_answer"])

1. 가상 문서 생성중...

 가상 문서: 북한에서 강제로 이루어지는 조직 생활은 주로 주민들을 정치적으로 통제하고 사회주의 이념을 주입하기 위한 수단으로 사용됩니다. 이러한 조직 생활의 대표적인 예로는 '인민반'과 '사회주의 애국청년동맹' 등이 있습니다.

1. **인민반**: 인민반은 북한 주민들이 속해 있는 가장 기초적인 사회 조직 단위로, 보통 20~30가구로 구성됩니다. 인민반장은 주민들의 일상생활을 감시하고, 당의 지시사항을 전달하며, 주민들의 사상 교육을 책임집니다. 인민반 회의는 정기적으로 열리며, 주민들은 회의에 참석하여 당의 방침을 학습하고, 서로의 생활을 감시하는 역할을 합니다.

2. **사회주의 애국청년동맹**: 이는 북한의 청년 조직으로, 만 14세부터 30세까지의 청년들이 가입 대상입니다. 이 조직은 청년들에게 사회주의 이념을 교육하고, 노동력 동원, 군사 훈련 등을 통해 체제에 대한 충성을 강화하는 역할을 합니다. 청년동맹은 정기적인 모임과 행사, 교육 프로그램을 통해 청년들의 사상과 행동을 통제합니다.

이 외에도 북한 주민들은 다양한 직장 단위나 농업 협동조합 등에서 조직 생활을 하며, 이러한 조직들은 모두 당의 지시를 충실히 이행하는 것을 목표로 하고 있습니다. 이러한 조직 생활은 개인의 사생활을 제한하고, 주민들을 철저히 감시하는 수단으로 작용하고 있습니다.

2. 실제 문서 검색중...

3. 최종 답변 생성중...

=== 프로세스 결과 ===

[가상 문서]
북한에서 강제로 이루어지는 조직 생활은 주로 주민들을 정치적으로 통제하고 사회주의 이념을 주입하기 위한 수단으로 사용됩니다. 이러한 조직 생활의 대표적인 예로는 '인민반'과 '사회주의 애국청년동맹' 등이 있습니다.

1. **인민반**: 인민반은 북한 주민들이 속해 있는 가장 기초적인 사회 조직 단위로, 보통 20~30가구로 구성됩니다. 인민반장은 주민들의 일상생활을 감시하고, 당의 지시사항을 전달하며, 주민들의 사상 교육을 책임집니다. 인민반 회의는 정기적으로 